In [8]:
import cv2
import numpy as np
from sklearn.cluster import DBSCAN


def preprocess_image(image_path, blur_ksize=(5, 5)):
    """
    Preprocess the input image by converting it to grayscale, applying Gaussian blur, and converting it to a binary image.
    
    Args:
        image_path (str): Path to the input image.
        blur_ksize (tuple): Kernel size for the Gaussian blur (default is (5, 5)).
        
    Returns:
        gray_blurred (numpy array): Blurred grayscale version of the image.
        original_img (numpy array): Original color image.
        gray (numpy array): Grayscale version of the image.
        binary_img (numpy array): Binary version of the image.
    """
    # Read the image
    original_img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if original_img is None:
        raise ValueError(f"Image at {image_path} not found.")
    
    # Convert to grayscale
    gray = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    gray_blurred = cv2.GaussianBlur(gray, blur_ksize, 0)
    _, binary_img = cv2.threshold(gray_blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return gray_blurred, original_img, gray, binary_img


def detect_circles(img, gray_blurred, questions, options):
    """
    Detect circles in the preprocessed grayscale image using the Hough Circle Transform.
    """
    for b in range(15, 5, -1): 
        for a in range(1, 30):  
            detected_circles = cv2.HoughCircles(gray_blurred, cv2.HOUGH_GRADIENT, 1, a, 
                            param1=b, param2=3*b, minRadius=a//2, maxRadius=a)
            if detected_circles is not None:
                detected_circles = np.uint16(np.around(detected_circles))
                if detected_circles.shape[1] == questions * options:
                    for pt in detected_circles[0, :]: 
                        a, b, r = pt[0], pt[1], pt[2] 
                        cv2.circle(img, (a, b), r, (0, 255, 0), 3)
                        cv2.circle(img, (a, b), 2, (0, 0, 255), 3)

                    cv2.imshow("Detected Circles", img)
                    cv2.imwrite("./testoutput/Detected_Circles.jpg", img)
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()
                    return detected_circles
    return None


def count_black_and_white_pixels(img, circles):
    """
    Count black and white pixels in the detected circles.
    """
    counts = []
    for (x, y, r) in circles:
        mask = np.zeros_like(img, dtype=np.uint8)
        cv2.circle(mask, (x, y), r, (255), thickness=-1)
        masked_area = cv2.bitwise_and(img, mask)
        black_pixels = np.sum(masked_area == 0)
        white_pixels = np.sum(masked_area == 255)
        counts.append((x, y, r, black_pixels, white_pixels))
    return counts


def group_by_row_dbscan(pixel_counts, eps=10):
    """
    Group circles by rows using DBSCAN clustering based on y-coordinates.
    """
    y_coords = np.array([[y] for (x, y, r, black_count, white_count) in pixel_counts])
    clustering = DBSCAN(eps=eps, min_samples=1).fit(y_coords)
    labels = clustering.labels_
    rows = {}
    for label, circle_data in zip(labels, pixel_counts):
        if label not in rows:
            rows[label] = []
        rows[label].append(circle_data)
    sorted_rows = sorted(rows.values(), key=lambda row: np.mean([c[1] for c in row]))
    sorted_rows = [sorted(row, key=lambda c: c[0]) for row in sorted_rows]
    return sorted_rows


def subtract_grouped_rows(grouped_rows_s, grouped_rows_t):
    """
    Subtract the black and white pixel values of grouped_rows_t from grouped_rows_s.
    """
    subtracted_rows = []
    for row_s, row_t in zip(grouped_rows_s, grouped_rows_t):
        subtracted_row = []
        for (x_s, y_s, r_s, black_s, white_s), (x_t, y_t, r_t, black_t, white_t) in zip(row_s, row_t):
            subtracted_black = black_s - black_t
            subtracted_white = white_s - white_t
            subtracted_row.append((x_s, y_s, r_s, subtracted_black, subtracted_white))
        subtracted_rows.append(subtracted_row)
    return subtracted_rows


def detect_marked_and_unmarked_bubbles(subtracted_grouped_rows, num_options, deviation_threshold=50):
    """
    Detect marked and unmarked bubbles based on white pixel values and a deviation threshold.
    """
    marked_bubbles = []
    for row in subtracted_grouped_rows:
        for question_start in range(0, len(row), num_options):
            question_group = row[question_start:question_start + num_options]
            white_pixel_values = [subtracted_white for (_, _, _, _, subtracted_white) in question_group]
            mean_white = np.mean(white_pixel_values)
            std_dev_white = np.std(white_pixel_values)
            if std_dev_white < deviation_threshold:
                marked_bubbles.append([0] * num_options)
            else:
                deviations = [abs(white_value - mean_white) for white_value in white_pixel_values]
                marked_question = np.zeros(num_options, dtype=int)
                max_deviation_index = np.argmax(deviations)
                marked_question[max_deviation_index] = 1
                marked_bubbles.append(marked_question)
    return np.array(marked_bubbles)


def calculate_score(answer_key, answer_student):
    """
    Compare the student's answers with the answer key and calculate the score.
    """
    total_score = 0
    result_per_question = []
    for key, student in zip(answer_key, answer_student):
        if np.array_equal(key, student):
            total_score += 1
            result_per_question.append(1)
        else:
            result_per_question.append(0)
    return total_score, result_per_question


questions=40
options=5

# Main processing sequence
gray_blurred_s, original_img_s, gray_s, binary_img_s = preprocess_image('./Test/2_S.jpg')
gray_blurred_t, original_img_t, gray_t, binary_img_t = preprocess_image('./Test/2_T.jpg')
gray_blurred_k, original_img_k, gray_k, binary_img_k = preprocess_image('./Test/2_K.jpg')

detected_circles_s = detect_circles(original_img_s, gray_blurred_s, questions, options)[0, :]
detected_circles_t = detect_circles(original_img_t, gray_blurred_t, questions, options)[0, :]
detected_circles_k = detect_circles(original_img_k, gray_blurred_k, questions, options)[0, :]

pixel_counts_s = count_black_and_white_pixels(binary_img_s, detected_circles_s)
pixel_counts_t = count_black_and_white_pixels(binary_img_t, detected_circles_t)
pixel_counts_k = count_black_and_white_pixels(binary_img_k, detected_circles_k)

grouped_rows_s = group_by_row_dbscan(pixel_counts_s, eps=10)
grouped_rows_t = group_by_row_dbscan(pixel_counts_t, eps=10)
grouped_rows_k = group_by_row_dbscan(pixel_counts_k, eps=10)

subtracted_grouped_rows = subtract_grouped_rows(grouped_rows_t, grouped_rows_s)
subtract_grouped_rows_key = subtract_grouped_rows(grouped_rows_t, grouped_rows_k)

answer_student = detect_marked_and_unmarked_bubbles(subtracted_grouped_rows, options)
answer_key = detect_marked_and_unmarked_bubbles(subtract_grouped_rows_key, options)

total_score, result_per_question = calculate_score(answer_key, answer_student)

# Print final results
print(f"Total Score: {total_score}")
for idx, result in enumerate(result_per_question, start=1):
    status = "Correct" if result == 1 else "Incorrect"
    print(f"Question {idx}: {status}")


TypeError: 'NoneType' object is not subscriptable